# Import libraries
Pandas - <br>
Numpy <br>
sklearn <br>
plotly <br>

In [1]:
import pandas as pd
import numpy as np
import hashlib
import os 
from utils import logger
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from utils import logger
import sys

# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d

import plotly.plotly as py
import plotly.graph_objs as go
import itertools as it

# %matplotlib notebook

# Feature Selection
#### Lasso
#### PCA - It is using the correlation between some dimensions and tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed. 
#### tSNE - looks at the original data that is entered into the algorithm and looks at how to best represent this data using less dimensions by matching both distributions

In [2]:
def lassoSelection(X_train, y_train, n):
    '''
    Lasso feature selection.  Select n features. 
    '''
    #lasso feature selection
    #print (X_train)
    clf = LassoCV(max_iter=10000,tol=0.001)
    sfm = SelectFromModel(clf,threshold=0)
    sfm.fit(X_train, y_train)
    X_transform = sfm.transform(X_train)
    n_features = X_transform.shape[1]

    # 	print("n_features=",n_features)
    #print(n_features)
    while n_features > n:
        sfm.threshold += 0.01
        X_transform = sfm.transform(X_train)
        n_features = X_transform.shape[1]
        print ("n_features =",n_features)
    features = [index for index,value in enumerate(sfm.get_support()) if value == True  ]
    logger.info("selected features are {}".format(features))
    logger.info("Features selected from Lasso using SelectFromModel with threshold {:6.4f}".format(sfm.threshold))
    return features

In [3]:
def pcaSelection(X_train, X_test, n):
    '''
    PCA feature selection.  Select n features. 
    '''
    pca = PCA(n_components=n)
    pca.fit(X_train)
    X_train_new = pca.transform(X_train)
    X_test_new = pca.transform(X_test)
    logger.info("X_train size after PCA: {}".format(X_train_new.shape))
    logger.info("X_test size after PCA: {}".format(X_test_new.shape))
    logger.info("Cumulative explained variation for {} principal components: {:6.4f}".format(n,np.sum(pca.explained_variance_ratio_)))
    return [X_train_new,X_test_new]

In [4]:
def tsneSelection(X_train, n, v):
    '''
    t-distributed Stochastic Neighbor Embedding feature selection.  Select n features. 
    '''
    tsne = TSNE(n_components=n,verbose=v)
    X_train_new = tsne.fit_transform(X_train)
    logger.info("X_train size after tSNE: {}".format(X_train_new.shape))
    return X_train_new

# Model

In [11]:
def model_fit_predict(X_train,X_test,y_train,y_test):

    # np.random.seed(2018)
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    # from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import precision_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import classification_report,confusion_matrix
    models = {
#         'LogisticRegression': LogisticRegression(random_state=0, multi_class='ovr',solver='lbfgs',max_iter=10000,tol=0.001),
#         'ExtraTreesClassifier': ExtraTreesClassifier(),
#         'RandomForestClassifier': RandomForestClassifier(),
#     #   	# 'AdaBoostClassifier': AdaBoostClassifier(),
#         # 'GradientBoostingClassifier': GradientBoostingClassifier(),
#         'SVC': SVC(decision_function_shape='ovo',max_iter=10000,tol=0.001),
        'MLP':MLPClassifier(solver='adam',max_iter=1000,verbose=1)
    }
    tuned_parameters = {
#         'LogisticRegression':{'C': [1, 10]},
#         # 'LogisticRegression':{'solver':['newton-cg','liblinear','sag'],'C': [1, 10]}
#         'ExtraTreesClassifier': { 'n_estimators': [100] },
#         'RandomForestClassifier': { 'n_estimators': [16, 32] },
#     #   	'AdaBoostClassifier': { 'n_estimators': [16, 32] },
#         # 'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.05] },
#         # 'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
#         'SVC': { 'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001] },
        'MLP':{ 'batch_size':[50,200],'alpha':[0.0001,0.005],'tol':[0.001,0.01,0.1],\
               'learning_rate':['constant', 'invscaling', 'adaptive'],'activation':['relu','logistic','tanh'],\
              'hidden_layer_sizes':[(30,30,30),(30,20,10),(100,50,10)]}
        #,'learning_rate':['constant', 'invscaling', 'adaptive'] 
    }
    scores= {}
    confusion_mat = {}
    report = {}
    for key in models:
        print("Running",key,"...")
        clf = GridSearchCV(models[key], tuned_parameters[key], scoring=None,  refit=True, cv=10)
        clf.fit(X_train,y_train)
        print(clf.best_params_)
        y_test_predict = clf.predict(X_test)
        precision = precision_score(y_test, y_test_predict,average='micro') # tp / (tp + fp)
        accuracy = accuracy_score(y_test, y_test_predict) #subset accuracy
        f1 = f1_score(y_test, y_test_predict,average='micro') # F1 = 2 * (precision * recall) / (precision + recall)
        recall = recall_score(y_test, y_test_predict, average='macro') 
        # specificity = specificity_score(y_test, y_test_predict)
        scores[key] = [precision,accuracy,f1,recall]
        confusion_mat[key]=confusion_matrix(y_test,y_test_predict)
        report[key]= classification_report(y_test,y_test_predict)
        print(scores[key])
        print(confusion_mat[key])
        print(report[key])
    return scores, confusion_mat, report

# Scatterplot Function

In [7]:
# Scatter plot 2D & 3D
num_class = 36
colors = it.cycle(["aquamarine", "crimson", "darkseagreen", "deeppink","wheat","violet","fuchsia","turquoise",\
                   "ivory", "honeydew", "rosybrown","red","lemonchiffon","darkorchid","mintcream","papayawhip",\
                   "beige","darkcyan","firebrick","deepskyblue","seashell","mediumpurple","goldenrod","lightcoral",\
                   "limegreen","cadetblue","darkmagenta","ghostwhite","gainsboro","paleturquoise","teal","peru",\
                  "maroon","olivedrab","springgreen","yellowgreen"])
classes = it.cycle(['Breast', 'Uterine Corpus', 'Head', 'Kidney Renal Clear', 'Lung Adenocarcinoma', 'Brain', 'Thyroid', 'Prostate', 'Ovarian', 'Lung Squamous', 'Skin', 'Colon', 'Stomach', 'Bladder', 'Liver', 'Cervical', 'Kidney Renal Papillary', 'Leukemia', 'Sarcoma', 'Esophageal', 'Pheochromocytoma', 'Pancreatic', 'Rectum', 'Testicular', 'Wilms', 'Thymoma', 'Mesothelioma', 'Adrenocortical', 'Uveal', 'Kidney Chromophobe', 'Uterine Carcinosarcoma', 'Lymphoid', 'Rhabdoid', 'Cholangiocarcinoma', 'Glioblastoma'])


def scatter2D(X_train_2d):
    '''
    Function to genrate traces for 2D scatter plot
    Args: 2-feature X_train of dimension [?,2]
    Return: list of scatter plot trace objects
    '''
    data=[]
    for label in range(0,num_class):
        filtered_idx = np.argwhere(y_train==label)[:,0]
        trace = go.Scatter(
            x=X_train_2d[filtered_idx,0],
            y=X_train_2d[filtered_idx,1],
            mode='markers',
            marker=dict(
                size=5,
                line=dict(
                    color=next(colors),
                    width=0.1
                    ),
                opacity=0.5
                ),
            name=next(classes)
            )
        data.append(trace)
    return data


def scatter3D(X_train_3d):
    '''
    Function to generate traces for 3D scatter plot
    Args: 3-feature X_train of dimension [?,3]
    ReturnL list of scatter plot trace objects
    '''
    data=[]
    for label in range(0,num_class):
        filtered_idx = np.argwhere(y_train==label)[:,0]
        trace = go.Scatter3d(
            x=X_train_3d[filtered_idx,0],
            y=X_train_3d[filtered_idx,1],
            z=X_train_3d[filtered_idx,2],
            mode='markers',
            marker=dict(
                size=5,
                line=dict(
                    color=next(colors),
                    width=0.1
                    ),
                opacity=0.5
                ),
            name=next(classes)
            )
        data.append(trace)
    return data

# Pre-processing
### Separate X (features) and y (labels)
### Split training (70%) and testing (30%) dataset
### Standardize data -- scale features such that they are 1) zero-mean, 2) one-variance

In [12]:
data_file = "../data/miRNA.csv" # directory to miRNA_matrix.csv

# Get dataset from csv
df = pd.read_csv(data_file)
y_data = df.pop('label').values
df.pop('file_id')
columns =df.columns
X_data = df.values
num_features_orig = X_data.shape[1]
logger.info("Original dataset size: {}".format(X_data.shape[0]))
logger.info("Total feature num: {}".format(num_features_orig))

# split the data to train and test set
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0)


logger.info("Training dataset size: {}".format(X_train.shape[0]))
logger.info("Testing dataset size: {}".format(X_test.shape[0]))
# # standardize the data (zero-mean,uniform variance)
scaler = StandardScaler().fit(X_train.astype(np.float64))
X_train = scaler.transform(X_train.astype(np.float64))
X_test = scaler.transform(X_test.astype(np.float64))
logger.info("Mean of X-data features before standardization: {:6.4f}".format(sum(X_data.mean(axis=0))/num_features_orig))
logger.info("STD of X-data features before standardization: {:6.4f}".format(sum(X_data.std(axis=0))/num_features_orig))
logger.info("Mean of X-train features after standardization: {:6.4f}".format(sum(X_train.mean(axis=0))/num_features_orig))
logger.info("STD of X-train features after standardization: {:6.4f}".format(sum(X_train.std(axis=0))/num_features_orig))
logger.info("Mean of X-test features after standardization: {:6.4f}".format(sum(X_test.mean(axis=0))/num_features_orig))
logger.info("STD of X-test features after standardization: {:6.4f}".format(sum(X_test.std(axis=0))/num_features_orig))

# Define Graph layout
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
        )
)

[2018-10-20 22:17:26,808 - GDC - INFO] Original dataset size: 11486
[2018-10-20 22:17:26,811 - GDC - INFO] Total feature num: 1881
[2018-10-20 22:17:27,910 - GDC - INFO] Training dataset size: 8040
[2018-10-20 22:17:27,911 - GDC - INFO] Testing dataset size: 3446
[2018-10-20 22:17:28,678 - GDC - INFO] Mean of X-data features before standardization: 2458.7368
[2018-10-20 22:17:28,935 - GDC - INFO] STD of X-data features before standardization: 4450.1113
[2018-10-20 22:17:28,957 - GDC - INFO] Mean of X-train features after standardization: -0.0000
[2018-10-20 22:17:29,089 - GDC - INFO] STD of X-train features after standardization: 0.9351
[2018-10-20 22:17:29,098 - GDC - INFO] Mean of X-test features after standardization: -0.0069
[2018-10-20 22:17:29,152 - GDC - INFO] STD of X-test features after standardization: 0.9125


# Feature Reduction/Selection
### Lasso
### PCA
### PCA + tSNE

In [ ]:
# LASSO feature selection
n = 50
feaures_columns = lassoSelection(X_train, y_train, n)
# feaures_columns = [25, 92, 119, 163, 166, 168, 181, 187, 194, 216, 240, 241, 248, \
# 253, 271, 272, 273, 282, 285, 287, 295, 305, 306, 336, 337, 339, 341, 351, 352, 488, \
# 495, 503, 511, 544, 588, 593, 641, 764, 1063, 1090, 1100, 1126, 1395, 1461, 1509, 1523, 1834, 1848, 1872]
scores_lasso, mat_lasso, report_lasso = model_fit_predict(X_train[:,feaures_columns],X_test[:,feaures_columns],y_train,y_test)

In [ ]:
# PCA feature reduction to n-components
n = 50 # 650 for ~80% representation
X_train_pca, X_test_pca = pcaSelection(X_train, X_test, n)
scores_pca, mat_pca, report_pca = model_fit_predict(X_train_pca,X_test_pca,y_train,y_test)

[2018-10-20 22:17:31,069 - GDC - INFO] X_train size after PCA: (8040, 50)
[2018-10-20 22:17:31,071 - GDC - INFO] X_test size after PCA: (3446, 50)
[2018-10-20 22:17:31,073 - GDC - INFO] Cumulative explained variation for 50 principal components: 0.3774


Running MLP ...
Iteration 1, loss = 2.81968278
Iteration 2, loss = 1.42519410
Iteration 3, loss = 0.93225344
Iteration 4, loss = 0.75905413
Iteration 5, loss = 0.65600621
Iteration 6, loss = 0.59203350
Iteration 7, loss = 0.54625957
Iteration 8, loss = 0.51423991
Iteration 9, loss = 0.48581418
Iteration 10, loss = 0.46324363
Iteration 11, loss = 0.44561754
Iteration 12, loss = 0.42706622
Iteration 13, loss = 0.41420686
Iteration 14, loss = 0.39838385
Iteration 15, loss = 0.39134233
Iteration 16, loss = 0.38001257
Iteration 17, loss = 0.36862568
Iteration 18, loss = 0.35820054
Iteration 19, loss = 0.35114707
Iteration 20, loss = 0.34077508
Iteration 21, loss = 0.33570811
Iteration 22, loss = 0.32612370
Iteration 23, loss = 0.31899874
Iteration 24, loss = 0.32239101
Iteration 25, loss = 0.30885602
Iteration 26, loss = 0.30188991
Iteration 27, loss = 0.29117171
Iteration 28, loss = 0.29102106
Iteration 29, loss = 0.28091097
Iteration 30, loss = 0.27735016
Iteration 31, loss = 0.27934379
I

Iteration 55, loss = 0.18571084
Iteration 56, loss = 0.17876715
Iteration 57, loss = 0.17769641
Iteration 58, loss = 0.17621993
Iteration 59, loss = 0.17929280
Iteration 60, loss = 0.16974192
Iteration 61, loss = 0.16813854
Iteration 62, loss = 0.16496425
Iteration 63, loss = 0.17473752
Iteration 64, loss = 0.16204637
Iteration 65, loss = 0.15963514
Iteration 66, loss = 0.16750260
Iteration 67, loss = 0.16305578
Iteration 68, loss = 0.15625306
Iteration 69, loss = 0.15506385
Iteration 70, loss = 0.15340717
Iteration 71, loss = 0.16271049
Iteration 72, loss = 0.15945424
Iteration 73, loss = 0.15054526
Iteration 74, loss = 0.14769013
Iteration 75, loss = 0.14445686
Iteration 76, loss = 0.14054868
Iteration 77, loss = 0.14877278
Iteration 78, loss = 0.13687193
Iteration 79, loss = 0.13302867
Iteration 80, loss = 0.13811898
Iteration 81, loss = 0.13849063
Iteration 82, loss = 0.14910658
Iteration 83, loss = 0.14091808
Iteration 84, loss = 0.13265380
Iteration 85, loss = 0.12400411
Iteratio

Iteration 133, loss = 0.07946766
Iteration 134, loss = 0.08525051
Iteration 135, loss = 0.08342941
Iteration 136, loss = 0.07648920
Iteration 137, loss = 0.07965407
Iteration 138, loss = 0.07880708
Iteration 139, loss = 0.08016811
Iteration 140, loss = 0.08838889
Iteration 141, loss = 0.08243313
Iteration 142, loss = 0.09938512
Iteration 143, loss = 0.08369690
Iteration 144, loss = 0.07492380
Iteration 145, loss = 0.06892625
Iteration 146, loss = 0.06817046
Iteration 147, loss = 0.06712388
Iteration 148, loss = 0.06874722
Iteration 149, loss = 0.07383901
Iteration 150, loss = 0.07370275
Iteration 151, loss = 0.07315343
Iteration 152, loss = 0.06528120
Iteration 153, loss = 0.06186524
Iteration 154, loss = 0.06264333
Iteration 155, loss = 0.06221224
Iteration 156, loss = 0.05943663
Iteration 157, loss = 0.05462425
Iteration 158, loss = 0.06057685
Iteration 159, loss = 0.07084276
Iteration 160, loss = 0.07441363
Iteration 161, loss = 0.12357545
Iteration 162, loss = 0.08022522
Iteration 

Iteration 92, loss = 0.12826945
Iteration 93, loss = 0.12358282
Iteration 94, loss = 0.13014136
Iteration 95, loss = 0.12196465
Iteration 96, loss = 0.12857770
Iteration 97, loss = 0.12528638
Iteration 98, loss = 0.11992136
Iteration 99, loss = 0.12038396
Iteration 100, loss = 0.12573746
Iteration 101, loss = 0.11968606
Iteration 102, loss = 0.11527773
Iteration 103, loss = 0.11689608
Iteration 104, loss = 0.11145958
Iteration 105, loss = 0.11188901
Iteration 106, loss = 0.11280054
Iteration 107, loss = 0.11567813
Iteration 108, loss = 0.11430515
Iteration 109, loss = 0.10766705
Iteration 110, loss = 0.10685600
Iteration 111, loss = 0.11074118
Iteration 112, loss = 0.10672836
Iteration 113, loss = 0.10625217
Iteration 114, loss = 0.09777368
Iteration 115, loss = 0.09443226
Iteration 116, loss = 0.09540739
Iteration 117, loss = 0.09514578
Iteration 118, loss = 0.10296053
Iteration 119, loss = 0.09287551
Iteration 120, loss = 0.09646036
Iteration 121, loss = 0.09987498
Iteration 122, los

...

Iteration 154, loss = 0.13719074
Iteration 155, loss = 0.13404741
Iteration 156, loss = 0.13501210
Iteration 157, loss = 0.13355895
Iteration 158, loss = 0.13672789
Iteration 159, loss = 0.13483875
Iteration 160, loss = 0.13303066
Iteration 161, loss = 0.13303530
Iteration 162, loss = 0.13134162
Iteration 163, loss = 0.13002377
Iteration 164, loss = 0.13742311
Iteration 165, loss = 0.12851836
Iteration 166, loss = 0.13268221
Iteration 167, loss = 0.12596190
Iteration 168, loss = 0.12566865
Iteration 169, loss = 0.12705631
Iteration 170, loss = 0.12276617
Iteration 171, loss = 0.12422375
Iteration 172, loss = 0.12289721
Iteration 173, loss = 0.12528813
Iteration 174, loss = 0.12124728
Iteration 175, loss = 0.12092590
Iteration 176, loss = 0.12115493
Iteration 177, loss = 0.12379745
Iteration 178, loss = 0.11947663
Iteration 179, loss = 0.11834040
Iteration 180, loss = 0.11581463
Iteration 181, loss = 0.11381959
Iteration 182, loss = 0.11711916
Iteration 183, loss = 0.11416590
Iteration 

Iteration 1, loss = 3.52274018
Iteration 2, loss = 2.73941541
Iteration 3, loss = 2.00723626
Iteration 4, loss = 1.52255778
Iteration 5, loss = 1.22063672
Iteration 6, loss = 1.02286016
Iteration 7, loss = 0.89108555
Iteration 8, loss = 0.79471294
Iteration 9, loss = 0.72450671
Iteration 10, loss = 0.67404678
Iteration 11, loss = 0.62988352
Iteration 12, loss = 0.59506441
Iteration 13, loss = 0.56168934
Iteration 14, loss = 0.53934246
Iteration 15, loss = 0.51706899
Iteration 16, loss = 0.49974850
Iteration 17, loss = 0.48500571
Iteration 18, loss = 0.46835301
Iteration 19, loss = 0.45628839
Iteration 20, loss = 0.44470013
Iteration 21, loss = 0.43289937
Iteration 22, loss = 0.42626585
Iteration 23, loss = 0.41474022
Iteration 24, loss = 0.40631992
Iteration 25, loss = 0.39665709
Iteration 26, loss = 0.39028758
Iteration 27, loss = 0.38367445
Iteration 28, loss = 0.37846093
Iteration 29, loss = 0.37689197
Iteration 30, loss = 0.36575436
Iteration 31, loss = 0.36428987
Iteration 32, los

Iteration 10, loss = 0.67420777
Iteration 11, loss = 0.63809664
Iteration 12, loss = 0.60801453
Iteration 13, loss = 0.58263617
Iteration 14, loss = 0.55558300
Iteration 15, loss = 0.53596736
Iteration 16, loss = 0.52081643
Iteration 17, loss = 0.50084479
Iteration 18, loss = 0.48698616
Training loss did not improve more than tol=0.100000 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 3.40557028
Iteration 2, loss = 2.61459512
Iteration 3, loss = 2.05405554
Iteration 4, loss = 1.62308991
Iteration 5, loss = 1.31623103
Iteration 6, loss = 1.11173792
Iteration 7, loss = 0.96748397
Iteration 8, loss = 0.87160767
Iteration 9, loss = 0.80206697
Iteration 10, loss = 0.74540953
Iteration 11, loss = 0.70279247
Iteration 12, loss = 0.66201224
Iteration 13, loss = 0.63073249
Iteration 14, loss = 0.60088844
Iteration 15, loss = 0.57714161
Iteration 16, loss = 0.55431869
Iteration 17, loss = 0.53720699
Iteration 18, loss = 0.52161187
Training loss did not improve more than tol=0.100000 fo

Iteration 61, loss = 0.26884176
Iteration 62, loss = 0.26076039
Iteration 63, loss = 0.25681830
Iteration 64, loss = 0.26224138
Iteration 65, loss = 0.26050021
Iteration 66, loss = 0.25603996
Iteration 67, loss = 0.25223760
Iteration 68, loss = 0.25222354
Iteration 69, loss = 0.25822414
Iteration 70, loss = 0.24407341
Iteration 71, loss = 0.24345867
Iteration 72, loss = 0.24037590
Iteration 73, loss = 0.23754210
Iteration 74, loss = 0.23732754
Iteration 75, loss = 0.23552492
Iteration 76, loss = 0.23883563
Iteration 77, loss = 0.23538344
Iteration 78, loss = 0.23037722
Iteration 79, loss = 0.22773423
Iteration 80, loss = 0.22483842
Iteration 81, loss = 0.23170544
Iteration 82, loss = 0.22434776
Iteration 83, loss = 0.22176836
Iteration 84, loss = 0.22268951
Iteration 85, loss = 0.23215344
Iteration 86, loss = 0.22167343
Iteration 87, loss = 0.21719587
Iteration 88, loss = 0.21454487
Iteration 89, loss = 0.21362147
Iteration 90, loss = 0.21579196
Iteration 91, loss = 0.21889449
Iteratio

Iteration 40, loss = 0.33690493
Iteration 41, loss = 0.33118643
Iteration 42, loss = 0.32629633
Iteration 43, loss = 0.32709077
Iteration 44, loss = 0.32283337
Iteration 45, loss = 0.32027369
Iteration 46, loss = 0.31700644
Iteration 47, loss = 0.31430484
Iteration 48, loss = 0.30719037
Iteration 49, loss = 0.30911943
Iteration 50, loss = 0.30225748
Iteration 51, loss = 0.29884524
Iteration 52, loss = 0.29829122
Iteration 53, loss = 0.29052864
Iteration 54, loss = 0.28619150
Iteration 55, loss = 0.29117549
Iteration 56, loss = 0.28360215
Iteration 57, loss = 0.28031744
Iteration 58, loss = 0.27920334
Iteration 59, loss = 0.27309161
Iteration 60, loss = 0.27602621
Iteration 61, loss = 0.27332693
Iteration 62, loss = 0.26642641
Iteration 63, loss = 0.26780425
Iteration 64, loss = 0.26539549
Iteration 65, loss = 0.26174620
Iteration 66, loss = 0.25978085
Iteration 67, loss = 0.25962904
Iteration 68, loss = 0.25445107
Iteration 69, loss = 0.25396096
Iteration 70, loss = 0.25376503
Iteratio

# Data Visualization

In [ ]:
# PCA 3-component scatter plot
X_train_pca3, X_test_pca3 = pcaSelection(X_train, X_test, 3)
pca3_traces = scatter3D(X_train_pca3)
fig1 = go.Figure(data=pca3_traces, layout=layout)
py.iplot(fig1, filename='PCA_3D_Scatter')

In [ ]:
# tSNE 3-component scatter plot (PCA->tSNE)
X_train_tsne3 = tsneSelection(X_train_pca,3,1)
tsne3_traces = scatter3D(X_train_tsne3)
fig3 = go.Figure(data=tsne3_traces, layout=layout)
py.iplot(fig3, filename='tSNE_3D_Scatter')

In [ ]:
# PCA 2-component scatter plot
X_train_pca2, X_test_pca2 = pcaSelection(X_train, X_test, 2)
pca2_traces = scatter2D(X_train_pca2)
fig2 = go.Figure(data=pca2_traces, layout=layout)
py.iplot(fig2, filename='PCA_2D_Scatter')

In [ ]:
# tSNE 2-component scatter plot (PCA->tSNE)
X_train_tsne2 = tsneSelection(X_train_pca,2,1)
tsne2_traces = scatter2D(X_train_tsne2)
fig4 = go.Figure(data=tsne2_traces, layout=layout)
py.iplot(fig4, filename='tSNE_2D_Scatter')

# Evalutation
## 